<a href="https://colab.research.google.com/github/MukundIsGod/SceneDetection/blob/master/mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
!pip uninstall -y tensorflow
!pip install tensorflow-2.4.1
!pip install tensorflow-gpu==2.0.0-beta0
!pip install tensorflow_hub
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
ERROR: Could not find a version that satisfies the requirement tensorflow-2.4.1 (from versions: none)
ERROR: No matching distribution found for tensorflow-2.4.1
     |████████████████████████████████| 349.7MB 51kB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 501kB 55.1MB/s 
     |████████████████████████████████| 3.1MB 48.8MB/s 


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_root='/content/drive/My Drive/MIT/indoorCVPR_09/Images'

In [ ]:
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)
print(TRAINING_DATA_DIR);
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    TRAINING_DATA_DIR, 
    subset="validation", 
    shuffle=True,
    target_size=IMAGE_SHAPE,
    batch_size=64
)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATA_DIR, 
    subset="training", 
    shuffle=True,
    target_size=IMAGE_SHAPE, 
    batch_size=64)

/content/drive/My Drive/MIT/indoorCVPR_09/Images
Found 3098 images belonging to 67 classes.
Found 12519 images belonging to 67 classes.


In [ ]:
model = tf.keras.Sequential([
  hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", 
                 output_shape=[1280],
                 trainable=False), 
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.6),
  tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])
model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(lr=0.001),
                  metrics=['accuracy'])

In [ ]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)
inc_history = model.fit_generator(train_generator, validation_data = valid_generator, steps_per_epoch = steps_per_epoch, epochs = 15, validation_steps=val_steps_per_epoch)

Epoch 1/15


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


98/98 [==============================] - 5956s 61s/step - loss: 3.0131 - accuracy: 0.1642 - val_loss: 1.5209 - val_accuracy: 0.5881
Epoch 2/15
98/98 [==============================] - 128s 1s/step - loss: 1.6641 - accuracy: 0.5068 - val_loss: 1.2261 - val_accuracy: 0.6604
Epoch 3/15
98/98 [==============================] - 123s 1s/step - loss: 1.3707 - accuracy: 0.5852 - val_loss: 1.1310 - val_accuracy: 0.6872
Epoch 4/15
98/98 [==============================] - 123s 1s/step - loss: 1.1960 - accuracy: 0.6485 - val_loss: 1.0729 - val_accuracy: 0.6963
Epoch 5/15
98/98 [==============================] - 124s 1s/step - loss: 1.0769 - accuracy: 0.6890 - val_loss: 1.0455 - val_accuracy: 0.6975
Epoch 6/15
98/98 [==============================] - 123s 1s/step - loss: 1.0193 - accuracy: 0.6935 - val_loss: 1.0345 - val_accuracy: 0.7021
Epoch 7/15
98/98 [==============================] - 123s 1s/step - loss: 0.9509 - accuracy: 0.7142 - val_loss: 1.0177 - val_accuracy: 0.7117
Epoch 8/15
98/98 [====

In [ ]:
final_loss, final_accuracy = model.evaluate(valid_generator, steps = val_steps_per_epoch)

49/49 [==============================] - 32s 650ms/step - loss: 1.3040 - accuracy: 0.6388


In [ ]:
print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))

Final loss: 1.30
Final accuracy: 63.88%


In [ ]:
!mkdir "saved_models"
saved_model_path = "./saved_models/mask_unmask.h5" 
model.save(saved_model_path)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("mobilenet_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Converted 262 variables to const ops.


INFO:tensorflow:Converted 262 variables to const ops.


9213488